In [ ]:
print("ok")

In [2]:
%pwd

'c:\\Users\\varsh\\OneDrive\\Desktop\\Projects\\Medical_chatbot\\research'

In [1]:
import os 
os.chdir("../")

In [2]:
%pwd

'c:\\Users\\varsh\\OneDrive\\Desktop\\Projects\\Medical_chatbot'

In [ ]:
%pwd

In [2]:
import os
os.chdir("../")

In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()

    return documents

In [6]:
extracted_data=load_pdf_file(data='Data/')

In [7]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5860


In [9]:
from langchain.embeddings import HuggingFaceEmbeddings

In [10]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

C:\Users\varsh\AppData\Local\Temp\ipykernel_17876\4238859041.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\varsh\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [7]:
query_result

[-0.03447733446955681,
 0.031023165211081505,
 0.006734927650541067,
 0.026108937337994576,
 -0.03936205059289932,
 -0.16030246019363403,
 0.06692402064800262,
 -0.006441427394747734,
 -0.0474504679441452,
 0.014758850447833538,
 0.07087534666061401,
 0.055527668446302414,
 0.01919332891702652,
 -0.026251325383782387,
 -0.01010952703654766,
 -0.026940423995256424,
 0.022307423874735832,
 -0.022226674482226372,
 -0.1496926248073578,
 -0.01749301142990589,
 0.007676245644688606,
 0.05435224249958992,
 0.0032543970737606287,
 0.03172587230801582,
 -0.08462141454219818,
 -0.029405992478132248,
 0.051595594733953476,
 0.048124048858881,
 -0.003314831294119358,
 -0.05827917903661728,
 0.04196928068995476,
 0.022210681810975075,
 0.1281888335943222,
 -0.02233896218240261,
 -0.011656305752694607,
 0.06292835623025894,
 -0.03287632763385773,
 -0.0912260189652443,
 -0.03117530234158039,
 0.05269957333803177,
 0.04703480377793312,
 -0.0842030942440033,
 -0.030056197196245193,
 -0.0207448247820138

In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [56]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GROQ_API_KEY=os.environ.get('GROQ_API_KEY')

In [17]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key="pcsk_24EYj4_7BaJnk1zr3sR9qJj8L7mgorvaH9xJMtGKKDxsXRkMP2cWRZLSCBZDddhTEUqefa")

index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [46]:
import os
os.environ["PINECONE_API_KEY"] = "pcsk_24EYj4_7BaJnk1zr3sR9qJj8L7mgorvaH9xJMtGKKDxsXRkMP2cWRZLSCBZDddhTEUqefa"
os.environ["OPENAI_API_KEY"] = "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"

In [23]:
from langchain.vectorstores import Pinecone

# Create the Pinecone vector store
docsearch = Pinecone.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name
)

AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



In [24]:
from langchain.vectorstores import Pinecone

# Embed each chunk and upsert the embeddings into your Pinecone index
docsearch = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)


In [25]:
docsearch

In [28]:
retriever= docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [29]:
retrieved_docs=retriever.invoke("What is Acne")

In [30]:
retrieved_docs

[Document(metadata={'page': 39.0, 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(metadata={'page': 39.0, 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(metadata={'page': 37.0, 'source': 'Data\\Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is\nthe most common skin disease. It affects nearly 17 million\npeople in the United States. While acne can arise at any')]

In [53]:
from langchain_openai import OpenAI
llm= OpenAI(temperature=0.4, max_tokens=200)

In [48]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate



system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human","{input}"),
    ]
)




In [54]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [52]:
# 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

ImportError: cannot import name 'RateLimitError' from 'langchain_core.runnables.base' (c:\Users\varsh\anaconda3\envs\medibot\lib\site-packages\langchain_core\runnables\base.py)